In [1]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함

!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 67 Dec 14 20:41 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=f1d959813bd3fa03a3f24cac95337082ae80667c608a18727ca0b3eebc192c25
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [2]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v


Kaggle API 1.5.6


In [3]:

! kaggle competitions download -c 2019-ml-finalproject

import zipfile
import os
os.mkdir('/content/input')

 54% 35.0M/65.0M [00:00<00:00, 108MB/s] 
100% 65.0M/65.0M [00:00<00:00, 222MB/s]


In [0]:
zip_ref = zipfile.ZipFile("/content/2019-ml-finalproject.zip", 'r')
zip_ref.extractall("/content/input")
zip_ref.close()

In [5]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-python-4.1.2.30
Uninstalling opencv-contrib-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/opencv_contrib_python-4.1.2.30.dist-info/*
Proceed (y/n)?   Successfully uninstalled opencv-contrib-python-4.1.2.30
     |████████████████████████████████| 25.0MB 1.4MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 30.6MB 1.5MB/s 


###############################################################################################################

In [7]:
!pip install kmc2

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.7MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252222 sha256=95dd47f8bdd57b686898f82d4df0f9013960b1995cb22fced74db1ff51ff1afa
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:

import pandas as pd
import numpy as np
import cv2
from sklearn.cluster import KMeans
import copy
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import GridSearchCV
import scipy.cluster.vq as vq
import kmc2

In [9]:
labelll=pd.read_csv('../content/input/Label2Names.csv')
labelll.loc[100] = ['1', 'Faces']
labelll.loc[101] = ['102', 'BACKGROUND_Google']

print(labelll.iloc[0][1])
print(labelll)

Faces_easy
       1              Faces
0      2         Faces_easy
1      3           Leopards
2      4         Motorbikes
3      5          accordion
4      6          airplanes
..   ...                ...
97    99      windsor_chair
98   100             wrench
99   101           yin_yang
100    1              Faces
101  102  BACKGROUND_Google

[102 rows x 2 columns]


In [10]:
#데이터 로드


data1 = ('../content/input/train')
data2 = ('../content/input/testAll_v2')

train_data = list()
train_name = list()
test_data = list()

namefile = os.listdir(data1)
testimgfile = os.listdir(data2)
testimgfile.sort()

#train
for file in namefile:
  imgfile = os.listdir(data1 + '/' + file)
  
  #(라벨링)
  for i in range (0,102):
    if (file == labelll.iloc[i][1]):
      label = labelll.iloc[i][0] #백그라운드 제외

  for img in imgfile:
    if (label == '102'):
      print("OK")
      break
    image = cv2.imread(data1 + '/' + file + '/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (256,256), interpolation = cv2.INTER_LINEAR)
    train_data.append(image)
    train_name.append(label)

train_data = np.array(train_data)
train_name = np.array(train_name)


#test
for img in testimgfile:
    image = cv2.imread(data2 + '/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (256,256), interpolation = cv2.INTER_LINEAR)
    test_data.append(image)

test_data = np.array(test_data)


print(train_data.shape)  #( , 100, 200)
print(train_name)




OK
(3030, 256, 256)
['58' '58' '58' ... '63' '63' '63']


In [0]:
def computeSIFT(data):
    x = []
    for i in range(0, len(data)):
        sift = cv2.xfeatures2d.SIFT_create()
        img = data[i]
        step_size = 8
        kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
        dense_feat = sift.compute(img, kp)
        x.append(dense_feat[1])
        
    return x

# extract dense sift features from training images
x_train = computeSIFT(train_data)
x_test = computeSIFT(test_data)

all_train_desc = []
for i in range(len(x_train)):
    for j in range(x_train[i].shape[0]):
        all_train_desc.append(x_train[i][j,:])

all_train_desc = np.array(all_train_desc)

In [12]:
print(x_train[7].shape)

(1024, 128)


In [0]:
def build_codebook(X, voc_size):
   
    features = np.vstack((descriptor for descriptor in X))
    seeding = kmc2.kmc2(features, voc_size)
    kmeans = MiniBatchKMeans(n_clusters=voc_size, init=seeding).fit(features)
    codebook = kmeans.cluster_centers_.squeeze()
    return codebook

In [0]:
codebook = build_codebook(all_train_desc, 800)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  """
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=800. Setting it to 3*k
  init_size=init_size)


In [0]:



def build_spatial_pyramid(image, descriptor, level):
  
    step_size = 8
   
    height = int(image.shape[0] / step_size)
    width = int(image.shape[1] / step_size)

    idx_crop = np.array(range(len(descriptor))).reshape(height,width)
    size = idx_crop.itemsize

   
    bh, bw = 2**(5-level), 2**(5-level)
    shape = (int(height/bh), int(width/bw), bh, bw)
    strides = size * np.array([width*bh, bw, width, 1])

    crops = np.lib.stride_tricks.as_strided(idx_crop, shape=shape, strides=strides)

    des_idxs = [col_block.flatten().tolist() for row_block in crops for col_block in row_block]

    pyramid = []
    for idxs in des_idxs:
        pyramid.append(np.asarray([descriptor[idx] for idx in idxs]))
    return pyramid

def spatial_pyramid_matching(image, descriptor, codebook, level):
    pyramid = []
    if level == 0:
        pyramid += build_spatial_pyramid(image, descriptor, level=0)
        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        return np.asarray(code).flatten()
    if level == 1:
        pyramid += build_spatial_pyramid(image, descriptor, level=0)
        pyramid += build_spatial_pyramid(image, descriptor, level=1)
        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        code_level_0 = 0.5 * np.asarray(code[0]).flatten()
        code_level_1 = 0.5 * np.asarray(code[1:]).flatten()
        return np.concatenate((code_level_0, code_level_1))
    if level == 2:
        pyramid += build_spatial_pyramid(image, descriptor, level=0)
        pyramid += build_spatial_pyramid(image, descriptor, level=1)
        pyramid += build_spatial_pyramid(image, descriptor, level=2)
        code = [input_vector_encoder(crop, codebook) for crop in pyramid]
        code_level_0 = 0.25 * np.asarray(code[0]).flatten()
        code_level_1 = 0.25 * np.asarray(code[1:5]).flatten()
        code_level_2 = 0.5 * np.asarray(code[5:]).flatten()
        return np.concatenate((code_level_0, code_level_1, code_level_2))

def input_vector_encoder(feature, codebook):
   
    code, _ = vq.vq(feature, codebook)
    word_hist, bin_edges = np.histogram(code, bins=range(codebook.shape[0] + 1), normed=True)
    return word_hist





In [16]:
x_train = [spatial_pyramid_matching(train_data[i],x_train[i], codebook, level=2) for i in range(len(train_data))]

x_test = [spatial_pyramid_matching(test_data[i], x_test[i], codebook, level=2) for i in range(len(test_data))]


x_train = np.asarray(x_train)
x_test = np.asarray(x_test)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: VisibleDeprecationWarning: Passing `normed=True` on non-uniform bins has always been broken, and computes neither the probability density function nor the probability mass function. The result is only correct if the bins are uniform, when density=True will produce the same result anyway. The argument will be removed in a future version of numpy.


In [0]:
def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result

gramMatrix = histogramIntersection(x_train, x_train)
cclf = SVC(kernel='precomputed')
cclf.fit(gramMatrix, train_name)
predictMatrix = histogramIntersection(x_test, x_train)
predict = cclf.predict(predictMatrix)

############################################################################################

In [25]:
# numpy 를 Pandas 이용하여 결과 파일로 저장


import pandas as pd

d = {'Id': testimgfile, 'Category': predict}
df = pd.DataFrame(data=d)

print(df)

                  Id Category
0     image_0001.jpg       45
1     image_0002.jpg       89
2     image_0003.jpg        7
3     image_0004.jpg       34
4     image_0005.jpg       14
...              ...      ...
1687  image_1688.jpg       27
1688  image_1689.jpg       61
1689  image_1690.jpg       29
1690  image_1691.jpg       38
1691  image_1692.jpg       35

[1692 rows x 2 columns]


In [0]:
df.to_csv('sohyeon_project.csv',index=False, header=True)

In [27]:
! kaggle competitions submit -c 2019-ml-finalproject -f sohyeon_project.csv -m "BaekSohyeon" 

100% 29.6k/29.6k [00:00<00:00, 102kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 

0.31914


*   스텝15
*   kmeans빠르게
* SPM 없음



베이스

*   항목 추가
*   항목 추가

